<a href="https://colab.research.google.com/github/rajy4683/EIP4_Phase1_Final/blob/master/InceptionV3_Assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')

!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 



In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

from keras.layers import Dense, Conv2D, BatchNormalization, Activation



In [0]:
import tensorflow as tf

from tqdm import tqdm_notebook as tqdm

import tensorflow.contrib.eager as tfe
#tf.enable_eager_execution()

In [0]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git
from keras_contrib.callbacks import CyclicLR

In [0]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

In [0]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

In [0]:
one_hot_df.head()


In [0]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15)
train_df.shape, val_df.shape

In [0]:
from datetime import datetime
def get_curr_time():
    return int(datetime.utcnow().strftime("%s"))

model_name_itr = 'wrn2_widrn_acc_incept_'+str(get_curr_time())
gdrive_home_path="/content/gdrive/My Drive/"
train_csv=gdrive_home_path+"train_df_"+model_name_itr+".csv"
val_csv=gdrive_home_path+"val_df_"+model_name_itr+".csv"
json_file=gdrive_home_path+"json_"+model_name_itr+".json"
png_file=gdrive_home_path+"png_"+model_name_itr+".png"
weights_file=gdrive_home_path+"h5_"+model_name_itr+".h5"

print("Model-name:",model_name_itr)
print(train_csv,val_csv,json_file,png_file,weights_file)
train_df.to_csv(train_csv, index=False)
val_df.to_csv(val_csv, index=False)

# /content/gdrive/My Drive/train_df_wrn2_widrn_acc_1577202722.csv 
# /content/gdrive/My Drive/val_df_wrn2_widrn_acc_1577202722.csv 
# /content/gdrive/My Drive/json_wrn2_widrn_acc_1577202722.json
#  /content/gdrive/My Drive/png_wrn2_widrn_acc_1577202722.png 
#  /content/gdrive/My Drive/h5_wrn2_widrn_acc_1577202722.h5

In [0]:
#standardize_all_images(train_df)
#items = train_df.iloc[:,:]
#image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
#print(image.shape)


In [0]:
import keras
import numpy as np
from google.colab.patches import cv2_imshow
from tensorflow.python.keras.utils.data_utils import Sequence

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

#class PersonDataGenerator(keras.utils.Sequence):
class PersonDataGenerator(Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True,normalize=False,aug_flow=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.normalize = normalize
        self.on_epoch_end()
        self.aug_flow=aug_flow
        #print("Shuffle = ",self.shuffle)

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        #print(batch_slice)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        #print(items["image_path"])
        
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        if(self.aug_flow is not None):
            image = self.aug_flow.flow(image,shuffle=False).next()
        if(self.normalize == True):
            train_mean = np.mean(image, axis=(0,1,2))
            train_std = np.std(image, axis=(0,1,2))
            #print(train_mean, train_std)
            normalize = lambda x: ((x - train_mean) / train_std).astype('float32')
            image = normalize(image)


        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [0]:
output_weights = ["gender_output", "imagequality_ouput","age_output", "weight_output", "bag_output", "footwear_output","emotion_output", "pose_output"]
col_splits = [_gender_cols_, _imagequality_cols_,_age_cols_, _weight_cols_, _carryingbag_cols_, _footwear_cols_, _emotion_cols_, _bodypose_cols_]
def get_dist(train_df,equalize_classes=False):
    loss_weights = {}
    index=0
    for selector_column in col_splits:
        print(selector_column)
        count = []
        percentile = []
        for age_split in selector_column:
            count.append( train_df[selector_column][train_df[age_split] == 1].shape[0])
        #print(count, np.round((count/11537.0)*100.0, 2))

        max_val = np.max(count)
        total_count = np.float32(train_df.shape[0])
        #print(count, )
        count_weights= [np.round(max_val/current_val,3) for current_val in count]
        print(count_weights)
        print(np.round((np.asarray(count)/total_count)*100.0, 2))

        #print("Top Class:",selector_column[np.argmax(count)],"Max Count",np.max(count))
        #print("Bottom Class:",selector_column[np.argmin(count)])
        #weights_dist = dict(zip(selector_column, count_weights))
        
        #print(weights_dist)
        weights_vals_dist={}
        index_val=0
        for y in range(len(count_weights)):
            weights_vals_dist[y]=count_weights[y]
            print(weights_vals_dist[y],y)
            #index_val+=index_val
            
            #loss_weights[output_weights[index]]={x,y}
        loss_weights[output_weights[index]]=weights_vals_dist
        #if equalize_classes == True:
        #    expanded_df = equalize_classwise_dist(train_df, selector_column, count)
        #    train_df = train_df.append(expanded_df, ignore_index=True)
        index+=1
    #print(loss_weights)
    return train_df,loss_weights

In [0]:
_,loss_weights_train=get_dist(train_df, equalize_classes=False)
loss_weights_train

In [0]:
train_df.head()

In [0]:
def random_crop(img, random_crop_size):
    # Note: image_data_format is 'channel_last'
    assert img.shape[2] == 3
    height, width = img.shape[0], img.shape[1]
    dy, dx = random_crop_size
    x = np.random.randint(0, width - dx + 1)
    y = np.random.randint(0, height - dy + 1)
    return img[y:(y+dy), x:(x+dx), :]


def crop_generator(batches, crop_length):
    """Take as input a Keras ImageGen (Iterator) and generate random
    crops from the image batches generated by the original iterator.
    """
    while True:
        batch_x, batch_y = next(iter(batches))
        batch_crops = np.zeros((batch_x.shape[0], crop_length, crop_length, 3))
        for i in range(batch_x.shape[0]):
            batch_crops[i] = random_crop(batch_x[i], (crop_length, crop_length))
        yield (batch_crops, batch_y)

In [0]:
import numpy as np
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()
        if p_1 > p:
            return input_img
        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)
            if left + w <= img_w and top + h <= img_h:
                break
        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)
        input_img[top:top + h, left:left + w, :] = c
        return input_img
    return eraser

In [0]:
# create train and validation data generators
BATCH_SIZE=8
aug_gen = ImageDataGenerator(horizontal_flip=True, 
                             vertical_flip=False,
                             rotation_range=5,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=[0.5,1.5],
                             shear_range=0.2,
                             #zca_whitening=True,
                             brightness_range=[0.5,1.5],
                             preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=False))

train_gen = PersonDataGenerator(train_df, batch_size=BATCH_SIZE,normalize=True,aug_flow=aug_gen)
valid_gen = PersonDataGenerator(val_df, batch_size=BATCH_SIZE, shuffle=False,normalize=True)

In [0]:
CROP_LENGTH=64
train_crops = crop_generator(train_gen, CROP_LENGTH)
valid_crops = crop_generator(valid_gen, CROP_LENGTH)

In [0]:
train_gen[0]

In [0]:
def get_image_batch(data_df, batch_size=32, shuffle=True,normalize=True, selected_field='age_output'):
    new_batch = PersonDataGenerator(data_df, batch_size,shuffle, normalize)
    images, targets = next(iter(new_batch))
    num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
    labels = np.asarray([ np.argmax(targets['age_output'][pos]) for pos in range(len(targets['age_output'])) ])
    return images,labels, targets, len(images)

In [0]:
images, y_train, targets, len_train = get_image_batch(train_df, batch_size=32,normalize=True, shuffle=True)

In [0]:
images_test, y_test, targets_test, len_test = get_image_batch(val_df, batch_size=32,normalize=True, shuffle=False)

In [0]:
def print_mean_std_for_batch(datagen_process):
    image_val,target = next(iter(datagen_process))
    print(image_val.shape)
    print(np.mean(image_val.round(2), axis=(0,1,2)),np.std(image_val.round(2), axis=(0,1,2)) )
    #print(np.mean(image_val.round(2), axis=(0,1,2)),np.std(image_val.round(2), axis=(0,1,2)) )  


In [0]:
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

images_test, targets_test = next(iter(valid_gen))

print(num_units)
#print(np.mean(images.round(2), axis=(0,1,2)),np.std(images.round(2), axis=(0,1,2)) )
#print(np.mean(images_test.round(2), axis=(0,1,2)),np.std(images_test.round(2), axis=(0,1,2)) )

print_mean_std_for_batch(train_gen)
print_mean_std_for_batch(valid_gen)

In [0]:
num_units = {'bag': 3,
 'emotion': 4,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [0]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import time,math
############# Weights initializer #################
def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

In [0]:
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam,SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K

In [0]:
from keras.models import Model
from keras.layers import Input, Add, Activation, Dropout, Flatten, Dense
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K
from keras.callbacks import ModelCheckpoint, LearningRateScheduler


In [0]:
tf.compat.v1.disable_eager_execution()

In [0]:
from keras import applications

In [0]:
!pip install git+https://github.com/keras-team/keras-applications

In [0]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

In [0]:
# wrn_28_10.compile(
#     optimizer=SGD(lr=0.5),
#     loss=tf.keras.losses.CategoricalCrossentropy(),
#     # loss_weights=loss_weights, 
#     metrics=["accuracy"]
# )
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

input_tensor = Input(shape=(224, 224, 3))
base_model = InceptionV3(weights=None, include_top=False, input_tensor=input_tensor)
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
#x = Dense(128, activation='relu')(x)
output_layers = [Dense(num_units[name], activation="softmax", name=f"{name}_output")(x) for name in num_units.keys()]

#Dense(num_units['age'], activation="softmax", name="age_output")(x)




model = Model(inputs=base_model.input, outputs=output_layers)

model.summary()

In [0]:
#Min: 12.520438 0.57330394
#Min: 12.54081 0.586987
#Min: 12.628042 0.18920188

#Min: 7.8906326 1.3513402
#wrn_28_10=create_model()
model.compile(
    optimizer=SGD(lr=0.1),
    #,momentum=MOMENTUM, nesterov=True),
    #optimizer=SGD(lr=0.01191919191919192),
    loss=tf.keras.losses.CategoricalCrossentropy(),     
    #metrics=["accuracy"]
    weighted_metrics=["accuracy"]
)


lr_finder = LRFinder_new(model)
lr_finder.find_generator(train_gen,start_lr=0.001, end_lr=0.2,epochs=10,steps_per_epoch=20)#,class_weight=loss_weights_train)
#lr_finder.find_generator(train_gen,start_lr=0.0001, end_lr=1,epochs=10,steps_per_epoch=20)

print("#Max:", np.max(lr_finder.losses),lr_finder.lrs[np.argmax(lr_finder.losses)])
print("#Min:", np.min(lr_finder.losses), lr_finder.lrs[np.argmin(lr_finder.losses)])

In [0]:
print(lr_finder.get_best_lr(1,1,1))
print(lr_finder.get_best_lr(10,1,1))
print(lr_finder.get_best_lr(20,1,1))

In [0]:
np.min(lr_finder.losses)

In [0]:
print("Max:", np.max(lr_finder.losses),lr_finder.lrs[np.argmax(lr_finder.losses)])
print("Min:", np.min(lr_finder.losses), lr_finder.lrs[np.argmin(lr_finder.losses)])

In [0]:
lr_finder.plot_loss(n_skip_end=1)

In [0]:
#wrn_28_10=create_model()
from keras.models import load_model
# import the necessary packages
from keras.callbacks import BaseLogger
import matplotlib.pyplot as plt
import numpy as np
import json
import os

# class LossHistory(keras.callbacks.Callback):
# 	def on_train_begin(self, logs={}):
# 		print("Clearing saved content on training start")
# 		self.losses = []
# 		self.best = np.Inf

class TrainingMonitor(BaseLogger):
	def __init__(self, figPath, jsonPath=None, startAt=0, backup_hist=True):
		# store the output path for the figure, the path to the JSON
		# serialized file, and the starting epoch
		super(TrainingMonitor, self).__init__()
		self.figPath = figPath
		self.jsonPath = jsonPath
		self.startAt = startAt
		self.backup_hist = backup_hist
		print("JSON path:",self.jsonPath)

	def on_train_begin(self, logs={}):
		# initialize the history dictionary
		self.H = {}
		#self.losses = []

		# if the JSON history path exists, load the training history
		if self.jsonPath is not None:
			if os.path.exists(self.jsonPath) and (self.backup_hist == True):
				#self.H = json.loads(open(self.jsonPath).read())
				backup_file_name=self.jsonPath+str(get_curr_time())+"_backup"
				print("Backing up history file:",self.jsonPath," to:",backup_file_name)
				os.rename(self.jsonPath,backup_file_name) 

				# # check to see if a starting epoch was supplied
				# if self.startAt > 0:
				# 	# loop over the entries in the history log and
				# 	# trim any entries that are past the starting
				# 	# epoch
				# 	for k in self.H.keys():
				# 		self.H[k] = self.H[k][:self.startAt]

	def on_epoch_end(self, epoch, logs={}):
		# loop over the logs and update the loss, accuracy, etc.
		# for the entire training process
		for (k, v) in logs.items():
			l = self.H.get(k, [])
			l.append(float(v))
			self.H[k] = l

		# check to see if the training history should be serialized
		# to file
		if self.jsonPath is not None:
			f = open(self.jsonPath, "w")
			f.write(json.dumps(self.H))
			f.close()


import os
plotPath = png_file
jsonPath = json_file
print(plotPath,jsonPath)

In [0]:
from datetime import datetime
# Prepare model model saving directory.
import os
save_dir = os.path.join('/content/gdrive/', 'My Drive')

model_name = 'assignment5_%s_model.{epoch:03d}.h5' % (model_name_itr)
print(model_name)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# # Prepare callbacks for model saving and for learning rate adjustment.
# checkpoint = ModelCheckpoint(filepath=filepath,
#                              monitor='val_loss',
#                              verbose=1,
#                              save_best_only=True,
#                              mode='min')

In [0]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git
from keras_contrib.callbacks import CyclicLR

In [0]:
EPOCHS=50
#LEARNING_RATE=1.3513402*0.1
#LEARNING_RATE=0.18883973*0.1
#LEARNING_RATE=0.5037587*0.1
#LEARNING_RATE=3.07534*0.1
LEARNING_RATE=0.0029628598*0.1

test_y = np.linspace(0,EPOCHS,EPOCHS)
x=[0, (EPOCHS+1)//5, EPOCHS]
y=[LEARNING_RATE*0.01, LEARNING_RATE, LEARNING_RATE*0.01]
interp_lr = np.interp(test_y, x, y)
def one_lr_schedule(epoch):
    # if(epoch <= 15):
    #     print("lr:",interp_lr[epoch+84],epoch)
    #     return interp_lr[epoch+84]
    print("lr:",interp_lr[epoch],epoch)
    return interp_lr[epoch]



clr = CyclicLR(
    base_lr=LEARNING_RATE*0.01,
    max_lr=LEARNING_RATE,
    step_size=STEPS_PER_EPOCH*4,
    mode='triangular2')


#interp_values = np.interp(, [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])
lr_scheduler = LearningRateScheduler(one_lr_schedule)
callbacks = [checkpoint, 
             #lr_scheduler,
             clr,
             TrainingMonitor(figPath=plotPath,jsonPath=jsonPath,startAt=0)]
                                                      

In [0]:
from datetime import datetime
# Prepare model model saving directory.
import os
save_dir = os.path.join('/content/gdrive/', 'My Drive')

model_name = 'a5_%s_model.{epoch:03d}.h5' % (model_name_itr)
print(model_name)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

def generate_new_callbacks(steps_per_epoch=50,epoch_count=50,min_lr=0.00001, max_lr=0.1):
    local_clr = CyclicLR(base_lr=min_lr,
                            max_lr=max_lr,
                            step_size=steps_per_epoch*4,
                            mode='triangular')
    checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')
    training_mon = TrainingMonitor(figPath=plotPath,
                                   jsonPath=jsonPath,
                                   startAt=0)
    ########## Introduced after 2x100 Epochs
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                    patience=25, 
                                                    restore_best_weights=True) 
    print("Returning new callback array with steps_per_epoch=",steps_per_epoch,
          "min_lr=",min_lr,
          "max_lr=",max_lr,
          "epoch_count=",epoch_count
          )
    callback_array = [local_clr, checkpoint, training_mon, early_stop]
    return callback_array

a5_wrn2_widrn_acc_incept_1577473943_model.{epoch:03d}.h5


In [0]:
history = [0] * 200

In [0]:
del model

NameError: ignored

In [0]:
#del model
STEPS_PER_EPOCH=30
EPOCHS=100
LEARNING_RATE=0.0029628598*0.1

callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE)
#model = Model(inputs=Input(shape=(224,224,3)), outputs=output_layers)
model.compile(
    optimizer=SGD(lr=0.0009999997),
    #,momentum=MOMENTUM, nesterov=True),
    #optimizer=SGD(lr=0.01191919191919192),
    loss=tf.keras.losses.CategoricalCrossentropy(),     
    metrics=["accuracy"]
    #weighted_metrics=["accuracy"]
)


history[0]=model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=EPOCHS,
    verbose=1,
    callbacks=callbacks,
    steps_per_epoch=STEPS_PER_EPOCH
    #class_weight=loss_weights_train
)

In [0]:
#del model
STEPS_PER_EPOCH=100
EPOCHS=100
LEARNING_RATE=0.0029628598*0.1

callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE)
#model = Model(inputs=Input(shape=(224,224,3)), outputs=output_layers)
# model.compile(
#     optimizer=SGD(lr=0.0009999997),
#     #,momentum=MOMENTUM, nesterov=True),
#     #optimizer=SGD(lr=0.01191919191919192),
#     loss=tf.keras.losses.CategoricalCrossentropy(),     
#     metrics=["accuracy"]
#     #weighted_metrics=["accuracy"]
# )
model.load_weights('/content/gdrive/My Drive/a5_wrn2_widrn_acc_incept_1577473943_model.062.h5')

history[0]=model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=EPOCHS,
    verbose=1,
    callbacks=callbacks,
    steps_per_epoch=STEPS_PER_EPOCH
    #class_weight=loss_weights_train
)

JSON path: /content/gdrive/My Drive/json_wrn2_widrn_acc_incept_1577473943.json
Returning new callback array with steps_per_epoch= 100 min_lr= 2.9628598000000003e-06 max_lr= 0.00029628598 epoch_count= 100
Backing up history file: /content/gdrive/My Drive/json_wrn2_widrn_acc_incept_1577473943.json  to: /content/gdrive/My Drive/json_wrn2_widrn_acc_incept_1577473943.json1577476781_backup
Epoch 1/100
100/100 [==============================] - 41s 406ms/step - loss: 7.9917 - gender_output_loss: 0.6921 - image_quality_output_loss: 0.9668 - age_output_loss: 1.4459 - weight_output_loss: 1.0272 - bag_output_loss: 0.9280 - pose_output_loss: 0.9803 - footwear_output_loss: 1.0492 - emotion_output_loss: 0.9021 - gender_output_acc: 0.5687 - image_quality_output_acc: 0.5700 - age_output_acc: 0.3888 - weight_output_acc: 0.6188 - bag_output_acc: 0.5487 - pose_output_acc: 0.5850 - footwear_output_acc: 0.4513 - emotion_output_acc: 0.7225 - val_loss: 7.8057 - val_gender_output_loss: 0.6975 - val_image_qual

In [0]:
STEPS_PER_EPOCH=200
EPOCHS=50
LEARNING_RATE=0.0029628598*0.1

callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE)
#model = Model(inputs=Input(shape=(224,224,3)), outputs=output_layers)
# model.compile(
#     optimizer=SGD(lr=0.0009999997),
#     #,momentum=MOMENTUM, nesterov=True),
#     #optimizer=SGD(lr=0.01191919191919192),
#     loss=tf.keras.losses.CategoricalCrossentropy(),     
#     metrics=["accuracy"]
#     #weighted_metrics=["accuracy"]
# )
model.load_weights('/content/gdrive/My Drive/a5_wrn2_widrn_acc_incept_1577473943_model.073.h5')

history[0]=model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=EPOCHS,
    verbose=1,
    callbacks=callbacks,
    steps_per_epoch=STEPS_PER_EPOCH
    #class_weight=loss_weights_train
)

JSON path: /content/gdrive/My Drive/json_wrn2_widrn_acc_incept_1577473943.json
Returning new callback array with steps_per_epoch= 200 min_lr= 2.9628598000000003e-06 max_lr= 0.00029628598 epoch_count= 50
Backing up history file: /content/gdrive/My Drive/json_wrn2_widrn_acc_incept_1577473943.json  to: /content/gdrive/My Drive/json_wrn2_widrn_acc_incept_1577473943.json1577481547_backup
Epoch 1/50
200/200 [==============================] - 61s 307ms/step - loss: 7.9265 - gender_output_loss: 0.6826 - image_quality_output_loss: 0.9700 - age_output_loss: 1.4535 - weight_output_loss: 1.0139 - bag_output_loss: 0.9169 - pose_output_loss: 0.9296 - footwear_output_loss: 1.0320 - emotion_output_loss: 0.9281 - gender_output_acc: 0.5550 - image_quality_output_acc: 0.5637 - age_output_acc: 0.3887 - weight_output_acc: 0.6194 - bag_output_acc: 0.5644 - pose_output_acc: 0.6194 - footwear_output_acc: 0.4825 - emotion_output_acc: 0.6987 - val_loss: 7.7419 - val_gender_output_loss: 0.6811 - val_image_qualit

In [0]:
STEPS_PER_EPOCH=200
EPOCHS=50
LEARNING_RATE=0.0029628598*0.1

callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE)
#model = Model(inputs=Input(shape=(224,224,3)), outputs=output_layers)
# model.compile(
#     optimizer=SGD(lr=0.0009999997),
#     #,momentum=MOMENTUM, nesterov=True),
#     #optimizer=SGD(lr=0.01191919191919192),
#     loss=tf.keras.losses.CategoricalCrossentropy(),     
#     metrics=["accuracy"]
#     #weighted_metrics=["accuracy"]
# )
model.load_weights('/content/gdrive/My Drive/a5_wrn2_widrn_acc_incept_1577473943_model.049.h5')

history[0]=model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=EPOCHS,
    verbose=1,
    callbacks=callbacks,
    steps_per_epoch=STEPS_PER_EPOCH
    #class_weight=loss_weights_train
)

JSON path: /content/gdrive/My Drive/json_wrn2_widrn_acc_incept_1577473943.json
Returning new callback array with steps_per_epoch= 200 min_lr= 2.9628598000000003e-06 max_lr= 0.00029628598 epoch_count= 50
Backing up history file: /content/gdrive/My Drive/json_wrn2_widrn_acc_incept_1577473943.json  to: /content/gdrive/My Drive/json_wrn2_widrn_acc_incept_1577473943.json1577487049_backup
Epoch 1/50
200/200 [==============================] - 60s 300ms/step - loss: 7.9072 - gender_output_loss: 0.6694 - image_quality_output_loss: 0.9964 - age_output_loss: 1.4436 - weight_output_loss: 1.0277 - bag_output_loss: 0.9324 - pose_output_loss: 0.9027 - footwear_output_loss: 1.0089 - emotion_output_loss: 0.9261 - gender_output_acc: 0.5906 - image_quality_output_acc: 0.5544 - age_output_acc: 0.3912 - weight_output_acc: 0.6119 - bag_output_acc: 0.5600 - pose_output_acc: 0.6413 - footwear_output_acc: 0.4925 - emotion_output_acc: 0.7050 - val_loss: 7.7292 - val_gender_output_loss: 0.6699 - val_image_qualit

In [0]:
history[1]=model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=50,
    verbose=1,
    #class_weight=loss_weights_train
    callbacks =callbacks
)

In [0]:
history[2]=model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=50,
    verbose=1,
    #class_weight=loss_weights_train
    callbacks =callbacks
)

In [0]:
#wrn_28_10.save('wrn_level_4.h5')

In [0]:
from matplotlib import pyplot as plt
import math
from keras.callbacks import LambdaCallback
import keras.backend as K
import numpy as np


class LRFinder_new:
    """
    Plots the change of the loss function of a Keras model when the learning rate is exponentially increasing.
    See for details:
    https://towardsdatascience.com/estimating-optimal-learning-rate-for-a-deep-neural-network-ce32f2556ce0
    """

    def __init__(self, model):
        self.model = model
        self.losses = []
        self.lrs = []
        self.best_loss = 1e9

    def on_batch_end(self, batch, logs):
        # Log the learning rate
        lr = K.get_value(self.model.optimizer.lr)
        self.lrs.append(lr)

        # Log the loss
        loss = logs['loss']
        self.losses.append(loss)

        # Check whether the loss got too large or NaN
        if batch > 5 and (math.isnan(loss) or loss > self.best_loss * 4):
            print("")
            print("Training stopped due to high loss",loss)
            self.model.stop_training = True
            return

        if loss < self.best_loss:
            self.best_loss = loss

        # Increase the learning rate for the next batch
        lr *= self.lr_mult
        K.set_value(self.model.optimizer.lr, lr)

    def find(self, x_train, y_train, start_lr, end_lr, batch_size=64, epochs=1):
        # If x_train contains data for multiple inputs, use length of the first input.
        # Assumption: the first element in the list is single input; NOT a list of inputs.
        N = x_train[0].shape[0] if isinstance(x_train, list) else x_train.shape[0]

        # Compute number of batches and LR multiplier
        num_batches = epochs * N / batch_size
        self.lr_mult = (float(end_lr) / float(start_lr)) ** (float(1) / float(num_batches))
        # Save weights into a file
        self.model.save_weights('tmp.h5')

        # Remember the original learning rate
        original_lr = K.get_value(self.model.optimizer.lr)

        # Set the initial learning rate
        K.set_value(self.model.optimizer.lr, start_lr)

        callback = LambdaCallback(on_batch_end=lambda batch, logs: self.on_batch_end(batch, logs))

        self.model.fit(x_train, y_train,
                       batch_size=batch_size, epochs=epochs,
                       callbacks=[callback])

        # Restore the weights to the state before model fitting
        self.model.load_weights('tmp.h5')

        # Restore the original learning rate
        K.set_value(self.model.optimizer.lr, original_lr)

    def find_generator(self, generator, start_lr, end_lr, epochs=1, steps_per_epoch=None, **kw_fit):
        if steps_per_epoch is None:
            try:
                steps_per_epoch = len(generator)
            except (ValueError, NotImplementedError) as e:
                raise e('`steps_per_epoch=None` is only valid for a'
                        ' generator based on the '
                        '`keras.utils.Sequence`'
                        ' class. Please specify `steps_per_epoch` '
                        'or use the `keras.utils.Sequence` class.')
        self.lr_mult = (float(end_lr) / float(start_lr)) ** (float(1) / float(epochs * steps_per_epoch))

        # Save weights into a file
        self.model.save_weights('tmp.h5')

        # Remember the original learning rate
        original_lr = K.get_value(self.model.optimizer.lr)

        # Set the initial learning rate
        K.set_value(self.model.optimizer.lr, start_lr)

        callback = LambdaCallback(on_batch_end=lambda batch,
                                                      logs: self.on_batch_end(batch, logs))

        self.model.fit_generator(generator=generator,
                                 epochs=epochs,
                                 steps_per_epoch=steps_per_epoch,
                                 callbacks=[callback],
                                 **kw_fit)

        # Restore the weights to the state before model fitting
        self.model.load_weights('tmp.h5')

        # Restore the original learning rate
        K.set_value(self.model.optimizer.lr, original_lr)

    def plot_loss(self, n_skip_beginning=10, n_skip_end=5, x_scale='log'):
        """
        Plots the loss.
        Parameters:
            n_skip_beginning - number of batches to skip on the left.
            n_skip_end - number of batches to skip on the right.
        """
        plt.ylabel("loss")
        plt.xlabel("learning rate (log scale)")
        plt.plot(self.lrs[n_skip_beginning:-n_skip_end], self.losses[n_skip_beginning:-n_skip_end])
        plt.xscale(x_scale)
        plt.show()

    def plot_loss_change(self, sma=1, n_skip_beginning=10, n_skip_end=5, y_lim=(-0.01, 0.01)):
        """
        Plots rate of change of the loss function.
        Parameters:
            sma - number of batches for simple moving average to smooth out the curve.
            n_skip_beginning - number of batches to skip on the left.
            n_skip_end - number of batches to skip on the right.
            y_lim - limits for the y axis.
        """
        derivatives = self.get_derivatives(sma)[n_skip_beginning:-n_skip_end]
        lrs = self.lrs[n_skip_beginning:-n_skip_end]
        plt.ylabel("rate of loss change")
        plt.xlabel("learning rate (log scale)")
        plt.plot(lrs, derivatives)
        plt.xscale('log')
        plt.ylim(y_lim)
        plt.show()

    def get_derivatives(self, sma):
        assert sma >= 1
        derivatives = [0] * sma
        for i in range(sma, len(self.lrs)):
            derivatives.append((self.losses[i] - self.losses[i - sma]) / sma)
        return derivatives

    def get_best_lr(self, sma, n_skip_beginning=10, n_skip_end=5):
        derivatives = self.get_derivatives(sma)
        best_der_idx = np.argmax(derivatives[n_skip_beginning:-n_skip_end])
        return self.lrs[n_skip_beginning:-n_skip_end][best_der_idx]
